# Fine tuning 

In [1]:
from transformers import (Trainer, TrainingArguments, EarlyStoppingCallback,
                          DataCollatorForLanguageModeling, AutoModelForCausalLM, AutoTokenizer)
from datasets import load_dataset, Dataset
from typing import Optional, Tuple
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import accelerate
from huggingface_hub import notebook_login
import keras

2024-10-20 15:22:34.002734: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 15:22:34.014453: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 15:22:34.017947: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-20 15:22:34.027187: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-20 15:22:34.976418: W tensorflow/compiler/tf2

In [2]:
dataset = load_dataset('mrm8488/CHISTES_spanish_jokes', split='train')
dataset.set_format('pandas')
df = dataset.to_pandas()

In [3]:
df.head(10)

,id,text,keywords,funny,category
0,0,"- ¡Rápido, necesitamos sangre!\n- Yo soy 0 pos...",sangre,1,otros
1,1,- ¿Cuál es el mejor portero del mundial? \n- E...,"futbol,porteros",1,otros
2,2,El otro día unas chicas llamarón a mi puerta y...,"dinero,agua",1,otros
3,3,"- Andresito, ¿qué planeta va después de Marte?...",planetas,1,profesiones
4,4,- ¿Por qué Bob Esponja no va al gimnasio? \n- ...,"esponja,gimnasios",1,otros
5,5,Van dos ciegos y le dice uno al otro: \n- Ojal...,ciegos,1,otros
6,6,Noticia de última hora!! \n\nMuere una suegra ...,"canarias,coches,noticias",2,familia
7,7,"– Mamá, mamá, en el colegio dicen que estoy lo...","locos,sillas",1,familia
8,8,"– Mamá, mamá, ¿me haces un bocata de jamón?\n–...","madres,jamón",1,otros
9,9,- Qué pasa si te expulsan de cuatro univerdade...,"universitarios,universidades",1,otros


In [4]:
def generate_ngrams(text: str, n: int) -> list:
    words = text.split()
    return [' '.join(words[i:i + n]) for i in range(len(words) - n + 1)]

# Crear una nueva columna solo con bigramas
df['bigrams'] = df['text'].apply(lambda x: generate_ngrams(x, 2))

# Unimos los bigramas con el texto original para enriquecer los ejemplos
df['text_enriched'] = df.apply(
    lambda row: row['text'] + ' ' + ' '.join(row['bigrams']), axis=1
)

# Ver el primer ejemplo de text_enriched
print(df['text_enriched'][0])

- ¡Rápido, necesitamos sangre!
- Yo soy 0 positivo.
- Pues muy mal, necesitamos una mentalidad optimista. - ¡Rápido, ¡Rápido, necesitamos necesitamos sangre! sangre! - - Yo Yo soy soy 0 0 positivo. positivo. - - Pues Pues muy muy mal, mal, necesitamos necesitamos una una mentalidad mentalidad optimista.


In [5]:
# veo el type de df
print(type(df))

<class 'pandas.core.frame.DataFrame'>


## Preparando el conjunto de datos

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "DeepESP/gpt2-spanish-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model

/home/kmansilla/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [9]:
def preprocess_function(max_len):
    def _preprocess_function(examples):
        return tokenizer(
            examples['text_enriched'],
            max_length=max_len,
            truncation=True,
            padding='longest',
            return_tensors='pt'
        )
    return _preprocess_function

In [10]:
# Cargar y preparar el dataset enriquecido
dataset = Dataset.from_pandas(df[['text_enriched']])
dataset = dataset.remove_columns(
    [col for col in dataset.column_names if col != 'text_enriched'])

# Tokenización en paralelo
tokenized_dataset = dataset.map(preprocess_function(
    max_len=256), batched=True, num_proc=4)
tokenized_dataset = tokenized_dataset.remove_columns(
    [col for col in tokenized_dataset.column_names if col != 'input_ids'])

# División del dataset
tokenized_dataset = tokenized_dataset.train_test_split(train_size=0.70)

# Asegurar formato PyTorch
tokenized_dataset.set_format('torch')
tokenized_dataset

# Verificar las muestras tokenizadas
print(f"Entrenamiento: {len(tokenized_dataset['train'])} muestras")
print(f"Prueba: {len(tokenized_dataset['test'])} muestras")
print(tokenizer.decode(tokenized_dataset['train'][0]['input_ids']))

Map (num_proc=4): 100%|██████████| 2419/2419 [00:00<00:00, 3403.99 examples/s]


Entrenamiento: 1693 muestras
Prueba: 726 muestras
- Doctor, doctor, tiene que ayudarme! No se que me pasa que enseguida pierdo los nervios y me pongo a insultar a todo el mundo.
- Está bien. Cuéntame sobre el asunto.
- ¿Y qué cree que estoy haciendo, pedazo de imbécil? - Doctor, Doctor, doctor, doctor, tiene tiene que que ayudarme! ayudarme! No No se se que que me me pasa pasa que que enseguida enseguida pierdo pierdo los los nervios nervios y y me me pongo pongo a a insultar insultar a a todo todo el el mundo. mundo. - - Está Está bien. bien. Cuéntame Cuéntame sobre sobre el el asunto. asunto. - - ¿Y ¿Y qué qué cree cree que que estoy estoy haciendo, haciendo, pedazo pedazo de de imbécil?<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|e

In [11]:
epochs = 10
batch_size = 8
logging_steps = len(tokenized_dataset['train']) // batch_size

# Definimos los parámetros de entrenamiento
training_args = TrainingArguments(
    output_dir='./hf-gpt',
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    learning_rate=2e-5,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    logging_steps=logging_steps,
    report_to="tensorboard",  # Reportar métricas a TensorBoard
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

# Crear el callback de early stopping
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

# Definir el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback]
)

# tensorboard --logdir=./logs

Descripcion de los argumentos:
- `num_train_epochs`: Número de iteraciones de entrenamiento.
- `learning_rate`: Tasa de aprendizaje para el optimizador, Un valor más alto puede acelerar el entrenamiento, pero un valor demasiado alto puede hacer que el modelo no converja.
- `per_device_train_batch_size`: Tamaño del lote por dispositivo de entrenamiento.
- `per_device_eval_batch_size`: Tamaño del lote por dispositivo de evaluación.
- `weight_decay`: Tasa de decaimiento de los pesos, que ayuda a evitar el sobreajuste al añadir una penalización a los pesos grandes.
- `eval_strategy`: significa que evalua al final de cada epoch.
- `save_strategy`: significa que guarda el modelo al final de cada epoch.
- `load_best_model_at_end`: significa que carga el mejor modelo al final del entrenamiento.
- `logging_steps`: Cada cuántos pasos se imprime el log.

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.599300,1.966256
2,1.801700,1.767062
3,1.564200,1.710328
4,1.406000,1.687153
5,1.305300,1.676827
6,1.219500,1.675656
7,1.150600,1.677190
8,1.113700,1.677086
9,1.071300,1.678793


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=1908, training_loss=1.4687158786525767, metrics={'train_runtime': 14852.6445, 'train_samples_per_second': 1.14, 'train_steps_per_second': 0.143, 'total_flos': 7075306241261568.0, 'train_loss': 1.4687158786525767, 'epoch': 9.0})

In [13]:
# Guardamos el modelo (solo correr si se vuelve a entrenar el modelo)
# trainer.save_model('trained-gpt2-bigram')
# tokenizer.save_pretrained('trained-gpt2-bigram')

('trained-gpt2-bigram/tokenizer_config.json',
 'trained-gpt2-bigram/special_tokens_map.json',
 'trained-gpt2-bigram/vocab.json',
 'trained-gpt2-bigram/merges.txt',
 'trained-gpt2-bigram/added_tokens.json',
 'trained-gpt2-bigram/tokenizer.json')

In [5]:
# Correr para cargar el modelo pre-entrenado
model = AutoModelForCausalLM.from_pretrained('trained-gpt2-bigram')
tokenizer = AutoTokenizer.from_pretrained('trained-gpt2-bigram')

In [5]:
#subo a huggingface
notebook_login()

In [17]:
model.push_to_hub('kevmansilla/generate_jokes_bigram')
tokenizer.push_to_hub('kevmansilla/generate_jokes_bigram')
print('Modelo subido a Hugging Face')

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Modelo subido a Hugging Face


In [2]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StopOnRepetition(StoppingCriteria):
    def __init__(self, tokenizer, max_repeats=2):
        self.tokenizer = tokenizer
        self.max_repeats = max_repeats

    def __call__(self, input_ids, scores, **kwargs):
        decoded_text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        words = decoded_text.split()
        
        # Detectar si hay demasiadas repeticiones consecutivas
        if len(words) > 3 and len(set(words[-self.max_repeats:])) == 1:
            return True
        return False

def generate_joke_from_prompt(prompt: str, max_length: int = 100, temperature: float = 0.7) -> str:
    """
    Genera un chiste coherente a partir de un prompt usando el modelo entrenado.

    Args:
        prompt (str): Inicio o tema del chiste.
        max_length (int): Longitud máxima del texto generado.
        temperature (float): Controla la creatividad del modelo.

    Returns:
        str: El chiste generado.
    """
    # Cargar modelo y tokenizer
    device = "cuda" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained('trained-gpt2-bigram')
    model = AutoModelForCausalLM.from_pretrained('trained-gpt2-bigram').to(device)

    # Tokenizar el prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    # Crear el criterio de parada
    stopping_criteria = StoppingCriteriaList([StopOnRepetition(tokenizer)])

    # Generar chiste con parámetros optimizados
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=50,  # Limitar las opciones para cada palabra
        top_p=0.9,  # Nucleus sampling para mayor diversidad
        repetition_penalty=1.5,
        no_repeat_ngram_size=2,  # Evitar repeticiones de bigramas
        stopping_criteria=stopping_criteria,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decodificar el texto generado
    joke = tokenizer.decode(output[0], skip_special_tokens=True)

    return joke

# Ejemplo de uso
prompt = "Un doctor le dice a su paciente"
chiste = generate_joke_from_prompt(prompt, max_length=60)
print(f"Chiste generado: {chiste}")



/home/kmansilla/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/kmansilla/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/kmansilla/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask is not set and cannot be inferred from i

Chiste generado: Un doctor le dice a su paciente:
- Doctor, tengo un problema. Un médico me ha dicho que tiene usted una enfermedad venérea y no sé cómo se llama la enfermedad pero si lo sabe es porque el otro día vino al consultorio del cirujano para decirme que tenía problemas de corazón.
